In [ ]:
!pip install -U langchain-openai langchain
OPENAI_API_KEY= "YOUR_API_KEY"

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(api_key=OPENAI_API_KEY)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model #chaine basique

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [6]:
with_message_history.invoke(
    {"ability": "math", "input": "Qu'est-ce que l'hypothalamus?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content="Une structure dans le cerveau qui régule divers processus biologiques tels que la température corporelle et l'appétit.")

In [7]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "Reformule en une dixaine de mots."},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content="Régulateur biologique du cerveau pour la température, l'appétit et autres fonctions vitales.")

In [9]:
hist = get_session_history("abc123")

In [10]:
from pprint import pprint
pprint(hist)

ChatMessageHistory(messages=[HumanMessage(content="Qu'est-ce que l'hypothalamus?"), AIMessage(content="Une structure dans le cerveau qui régule divers processus biologiques tels que la température corporelle et l'appétit."), HumanMessage(content='Reformule en une dixaine de mots.'), AIMessage(content="Régulateur biologique du cerveau pour la température, l'appétit et autres fonctions vitales.")])
